In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.H2("Developed by Bibek Shrestha"),
    html.Hr(),
    html.Div(
        [
            html.Div(
                [
                    html.Label("Filter by Rescue Type:", style={'font-weight': 'bold'}),
                    dcc.RadioItems(
                        id='filter-type',
                        options=[
                            {'label': 'All', 'value': 'all'},
                            {'label': 'Rescue', 'value': 'rescue'},
                            {'label': 'Transfer', 'value': 'transfer'}
                        ],
                        value='all',
                        inline=True
                    ),
                ],
                style={'margin-bottom': '20px'}
            ),
            html.Div(
                [
                    html.Label("Filter by Preferred Dog Breed:", style={'font-weight': 'bold'}),
                    dcc.Dropdown(
                        id='filter-breed',
                        options=[{'label': 'All', 'value': 'all'}] + [{'label': breed, 'value': breed} for breed in sorted(df['breed'].unique())],
                        value='all',
                        multi=True
                    ),
                ],
                style={'margin-bottom': '20px'}
            ),
        ]
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        row_selectable='single',
        selected_rows=[0],
        sort_action='native',
        filter_action='native',
        style_data_conditional=[]
    ),
    html.Br(),
    html.Hr(),
    #sets up dashboard so chart and geolocation chart are side by side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-type', 'value'),
     Input('filter-breed', 'value')])
def update_dashboard(filter_type, filter_breed):
    query = {}
    if filter_type == 'rescue':
        query['outcome_type'] = 'Transfer'
    elif filter_type == 'transfer':
        query['outcome_type'] = 'Adoption' 

    if filter_breed != 'all':
        if isinstance(filter_breed, str):
            query['breed'] = filter_breed
        elif isinstance(filter_breed, list) and filter_breed:
            query['breed'] = {'$in': filter_breed}

    filtered_data = pd.DataFrame.from_records(db.read(query))
    filtered_data.drop(columns=['_id'], inplace=True, errors='ignore')
    data = filtered_data.to_dict('records')
    return data


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or not viewData:
        return html.P("No data to display for the chart.")
    dff = pd.DataFrame(viewData)
    breed_counts = dff['breed'].value_counts().nlargest(10).reset_index()
    breed_counts.columns = ['breed', 'count']
    fig = px.bar(breed_counts, x='breed', y='count', title='Top 10 Preferred Animal Breeds')
    return [
        dcc.Graph(
            id='breed-bar-chart',
            figure=fig
        )
    ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:29836/
